## Ridesharing Project

In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import math
import os
from pymoo.core.problem import Problem, ElementwiseProblem
from pymoo.core.population import Population
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.optimize import minimize
from pymoo.core.repair import Repair
from pymoo.core.crossover import Crossover
from pymoo.core.mutation import Mutation
from pymoo.core.sampling import Sampling
from pymoo.core.selection import Selection
from pymoo.util.misc import random_permuations
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from pymoo.factory import get_reference_directions, get_selection
from pymoo.core.individual import Individual

In [2]:
#Vehicle Type Fuel Capacity Emission Cost
vehicle_types = {
    1: {
        'type': 'mini car',
        'fuel': 'petrol',
        'capacity': '1',
        'emission': np.array([1, 2*0.9]),
        'cost': 1
    },
    3: {
        'type': 'standard car',
        'fuel': 'electric',
        'capacity': '3',
        'emission': np.array([1.0/3.4, 2.0*0.9/3.4, 
                              3.0*0.9**2/3.4, 4.0*0.9**3/3.4]),
        'cost': 3
    },
    6: {
        'type': 'standard car',
        'fuel': 'petrol',
        'capacity': '6',
        'emission': np.array([1, 2*0.9, 3*0.9**2, 4*0.9**3, 
                              5*0.9**4, 6*0.9**5, 7*0.9**6]),
        'cost': 6
    },
    9: {
        'type': 'family car',
        'fuel': 'petrol',
        'capacity': '10',
        'emission': np.array([1, 2*0.9, 3*0.9**2, 4*0.9**3, 5*0.9**4, 
                              6*0.9**5, 7*0.9**6, 8*0.9**7, 9*0.9**8, 10*0.9**9, 11*0.9**10]),
        'cost': 10
    },
    30: {
        'type': 'coach',
        'fuel': 'petrol',
        'capacity': '30',
        'emission': np.array([1/4, 2/4*0.9, 3/4*0.9**2, 4/4*0.9**3, 5/4*0.9**4, 6/4*0.9**5, 
                              7/4*0.9**6, 8/4*0.9**7, 9/4*0.9**8, 10/4*0.9**9, 11/4*0.9**10, 
                              12/4*0.9**11, 13/4*0.9**12, 14/4*0.9**13, 15/4*0.9**14, 16/4*0.9**15, 
                              17/4*0.9**16, 18/4*0.9**17, 19/4*0.9**18, 20/4*0.9**19, 21/4*0.9**20, 
                              22/4*0.9**21, 23/4*0.9**22, 24/4*0.9**23, 25/4*0.9**24, 26/4*0.9**25, 
                              27/4*0.9**26, 28/4*0.9**27, 29/4*0.9**28, 30/4*0.9**29, 31/4*0.9**30]),
        'cost': 30
    },
    60: {
        'type': 'coach',
        'fuel': 'petrol',
        'capacity': '30',
        'emission': np.array([1/4, 2/4*0.9, 3/4*0.9**2, 4/4*0.9**3, 5/4*0.9**4, 6/4*0.9**5, 
                              7/4*0.9**6, 8/4*0.9**7, 9/4*0.9**8, 10/4*0.9**9, 11/4*0.9**10, 
                              12/4*0.9**11, 13/4*0.9**12, 14/4*0.9**13, 15/4*0.9**14, 16/4*0.9**15, 
                              17/4*0.9**16, 18/4*0.9**17, 19/4*0.9**18, 20/4*0.9**19, 21/4*0.9**20, 
                              22/4*0.9**21, 23/4*0.9**22, 24/4*0.9**23, 25/4*0.9**24, 26/4*0.9**25, 
                              27/4*0.9**26, 28/4*0.9**27, 29/4*0.9**28, 30/4*0.9**29, 31/4*0.9**30,
                              32/4*0.9**31, 33/4*0.9**32, 34/4*0.9**33, 35/4*0.9**34, 36/4*0.9**35, 
                              37/4*0.9**36, 38/4*0.9**37, 39/4*0.9**38, 40/4*0.9**39, 41/4*0.9**40, 
                              42/4*0.9**41, 43/4*0.9**42, 44/4*0.9**43, 45/4*0.9**44, 46/4*0.9**45, 
                              47/4*0.9**46, 48/4*0.9**47, 49/4*0.9**48, 50/4*0.9**49, 51/4*0.9**50, 
                              52/4*0.9**51, 53/4*0.9**52, 54/4*0.9**53, 55/4*0.9**54, 56/4*0.9**55, 
                              57/4*0.9**56, 58/4*0.9**57, 59/4*0.9**58, 60/4*0.9**59, 61/4*0.9**60
                             ]),
        'cost': 60
    }
}


In [19]:
# size down the instance

file = 'original_instances/rs-mny-m1k-c3-d6-s10-x1.0.instance'
re_path = 'instances_sizedown/'

count = 0
ins_col = []
ins_to_use = []
ins_list = dict()

instance = open(file, 'r')
head = [next(instance) for x in range(4)]
name = head[0].replace('\n', '').split('-')
road_map_code = name[1]
if road_map_code == 'bj5':
    road_map = 'Beijing'
elif road_map_code == 'cd1':
    road_map = 'Chengdu'
else:
    road_map = 'Manhattan'
capacity = name[3].replace('c', '')
v_num = head[2].replace('\n', '').split(' ')[1]
r_num = head[3].replace('\n', '').split(' ')[1]
ins_index = road_map_code + '-' + v_num + '-' + r_num
if ins_index in ins_list.keys():
    ins_list[ins_index].append(capacity)
else:
    ins_list[ins_index] = [capacity]
    
inst = pd.read_csv(file, skiprows = lambda x: x in range(6), sep = '\t', header=None)
inst.columns = ['id', 's', 't', 'Q', 'time_s', 'time_t']
inst.head(5)
v_id = np.where(inst['Q'] < 0)[0][:10]
r_id = np.where(inst['Q'] > 0)[0]
count = 0
for i in range(10, 40, 10):  
    for tp in [1, 3, 9]:
        count += 1
        re_info = open(re_path + 'instance-info_'+ str(count) + '.txt', 'w')
        re_vehicles = re_path + 'instance_'+ str(count) + '-vehicles.csv'
        re_riders = re_path + 'instance_'+ str(count) + '-riders.csv'
        
        tmp_r_id = r_id[:i]
        data = {
            "index": inst.iloc[tmp_r_id]['id'] - r_id[0],
            "s": inst.iloc[tmp_r_id]['s'],
            "t": inst.iloc[tmp_r_id]['t'],
            "et": inst.iloc[tmp_r_id]['time_s']
        }
        riders = pd.DataFrame(data=data)
        riders.to_csv(re_riders, index=False)
        
        if tp == 'random_types':
            v_types = np.random.choice([1,3,6,9,30,60], len(v_id))
        else:
            v_types = np.full([len(v_id),], tp)
    
        data = {
            "index": inst.iloc[v_id]['id'],
            "s": inst.iloc[v_id]['s'],
            "t": inst.iloc[v_id]['s'], # since this data set does not set destination for vehicles, we assume they return to s
            "p": v_types,
            "e": list(map(lambda x: vehicle_types[x]['emission'], v_types)),
            "c": list(map(lambda x: vehicle_types[x]['cost'], v_types))
        }
        vehicles = pd.DataFrame(data=data)
        vehicles.to_csv(re_vehicles, index=False)
    
        info = road_map_code + '.edges\n' + re_riders + '\t' + str(i) + '\n' + re_vehicles + '\t' + str(len(v_id)) + '\n' + 'capacity:\t' + capacity + '\n' + 'original_instance:\t' + file
        re_info.write(info)
        re_info.close()
        
        ins_info = str(count) + ',' + road_map_code + ',' + str(i) + ',' + str(len(v_id)) + ',' + str(tp)
        print(ins_info)

instance.close()    
# ins_to_use is the instances file for our experiments


1,mny,10,10,1
2,mny,10,10,3
3,mny,10,10,9
4,mny,20,10,1
5,mny,20,10,3
6,mny,20,10,9
7,mny,30,10,1
8,mny,30,10,3
9,mny,30,10,9


In [40]:
# collect information about all instances
pre_path = 'original_instances/'
re_path = 'instances_sizecontrol/'
folders = os.listdir(pre_path)
folders.sort()
count = 0
ins_col = []
ins_to_use = []
ins_list = dict()
for a_ins in folders:
    fre = a_ins.split('-')[-1].split('.')[0]
    inst = a_ins.replace('d6','').replace('d12','').replace('d24','')
    if inst in ins_col:
        continue
    count += 1
    ins_col.append(inst)
    ins_to_use.append(a_ins)
    
    file = pre_path + a_ins
    instance = open(file, 'r')
    head = [next(instance) for x in range(4)]
    name = head[0].replace('\n', '').split('-')
    road_map_code = name[1]
    if road_map_code == 'bj5':
        road_map = 'Beijing'
    elif road_map_code == 'cd1':
        road_map = 'Chengdu'
    else:
        road_map = 'Manhattan'
    capacity = name[3].replace('c', '')
    v_num = head[2].replace('\n', '').split(' ')[1]
    r_num = head[3].replace('\n', '').split(' ')[1]
    ins_index = road_map_code + '-' + v_num + '-' + r_num
    if ins_index in ins_list.keys():
        ins_list[ins_index].append(capacity)
    else:
        ins_list[ins_index] = [capacity]
    
    inst = pd.read_csv(file, skiprows = lambda x: x in range(6), sep = '\t', header=None)
    inst.columns = ['id', 's', 't', 'Q', 'time_s', 'time_t']
    inst.head(5)
    v_id = np.where(inst['Q'] < 0)[0]
    r_id = np.where(inst['Q'] > 0)[0]
    #print("Number of vehicles: ", len(v_id))
    #print("Number of riders: ", len(r_id))
    
    re_info = open(re_path + 'instance-info_'+ str(count) + '.txt', 'w')
    re_vehicles = re_path + 'instance_'+ str(count) + '-vehicles.csv'
    re_riders = re_path + 'instance_'+ str(count) + '-riders.csv'
    
    data = {
        "index": inst.iloc[r_id]['id'] - r_id[0],
        "s": inst.iloc[r_id]['s'],
        "t": inst.iloc[r_id]['t'],
        "et": inst.iloc[r_id]['time_s']
    }
    riders = pd.DataFrame(data=data)
    riders.to_csv(re_riders, index=False)
    
    data = {
        "index": inst.iloc[v_id]['id'],
        "s": inst.iloc[v_id]['s'],
        "t": inst.iloc[v_id]['s'], # since this data set does not set destination for vehicles, we assume they return to s
        "p": 0 - inst.iloc[v_id]['Q'],
        "e": list(map(lambda x: vehicle_types[0-x]['emission'], inst.iloc[v_id]['Q'])),
        "c": list(map(lambda x: vehicle_types[0-x]['cost'], inst.iloc[v_id]['Q']))
    }
    vehicles = pd.DataFrame(data=data)
    vehicles.to_csv(re_vehicles, index=False)
    
    info = road_map_code + '.edges\n' + re_riders + '\t' + r_num + '\n' + re_vehicles + '\t' + v_num + '\n' + 'capacity:\t' + capacity + '\n' + 'original_instance:\t' + file
    re_info.write(info)
    re_info.close()

# ins_to_use is the instances file for our experiments

In [41]:
# simulate that vehicles are different types
count = 99
ins_extra = dict()
for a_ins in ins_to_use:
    file = pre_path + a_ins
    road_map_code = a_ins.split('-')[1]
    
    instance = open(file, 'r')
    head = [next(instance) for x in range(4)]
    road_map_code = head[1].split(' ')[0]
    v_num = head[2].replace('\n', '').split(' ')[1]
    r_num = head[3].replace('\n', '').split(' ')[1]
    ins_index = road_map_code + '-' + v_num + '-' + r_num
    if ins_index in ins_extra.keys():
        continue
    else:
        ins_list[ins_index].append('mix')
        ins_extra[ins_index] = True
    
    count += 1
    inst = pd.read_csv(file, skiprows = lambda x: x in range(6), sep = '\t', header=None)
    inst.columns = ['id', 's', 't', 'Q', 'time_s', 'time_t']
    inst.head(5)
    v_id = np.where(inst['Q'] < 0)[0]
    r_id = np.where(inst['Q'] > 0)[0]
    
    re_info = open(re_path + 'instance-info_'+ str(count) + '.txt', 'w')
    re_vehicles = re_path + 'instance_'+ str(count) + '-vehicles.csv'
    re_riders = re_path + 'instance_'+ str(count) + '-riders.csv'
    
    data = {
        "index": inst.iloc[r_id]['id'] - r_id[0],
        "s": inst.iloc[r_id]['s'],
        "t": inst.iloc[r_id]['t'],
        "et": inst.iloc[r_id]['time_s']
    }
    riders = pd.DataFrame(data=data)
    riders.to_csv(re_riders, index=False)
    
    v_id = np.where(inst['Q'] < 0)[0]
    random_types = np.random.choice([1,3,6,9,30,60], len(v_id))
    
    data = {
        "index": inst.iloc[v_id]['id'],
        "s": inst.iloc[v_id]['s'],
        "t": inst.iloc[v_id]['s'], # since this data set does not set destination for vehicles, we assume they return to s
        "p": random_types,
        "e": list(map(lambda x: vehicle_types[x]['emission'], random_types)),
        "c": list(map(lambda x: vehicle_types[x]['cost'], random_types))
    }
    vehicles = pd.DataFrame(data=data)
    vehicles.to_csv(re_vehicles, index=False)
    
    info = road_map_code + '.edges\n' + re_riders + '\t' + r_num + '\n' + re_vehicles + '\t' + v_num + '\n' + 'capacity:\t random_mix' + '\n' + 'original_instance:\t' + file
    re_info.write(info)
    re_info.close()
    
# ins_to_use is the instances file for our experiments

In [42]:
instances_dict = dict()
for file in os.listdir("datasets/info/"):
    if '.txt' not in file:
        continue
    index = file.split('_')[1].replace('.txt', '')
    f = open("datasets/info/" + file, 'r')
    info_text = [next(f) for x in range(4)]
    road_map = info_text[0].replace('\n', '')
    n_vehicles = info_text[1].replace('\n', '').split('\t')[1]
    n_riders = info_text[2].replace('\n', '').split('\t')[1]
    cap = info_text[3].replace('\n', '').split('\t')[1]
    if '9' in cap:
        cap = 8
    instances_dict[int(index)] = index + ',' + road_map + ',' + n_vehicles + ',' + n_riders + ',' + str(cap)
    
for key in sorted(instances_dict):
    print(instances_dict[key])

1,bj5.edges,17467,10000,1
2,bj5.edges,17467,10000,3
3,bj5.edges,8754,10000,3
4,bj5.edges,34152,10000,3
5,bj5.edges,65500,10000,3
6,bj5.edges,17467,10000,6
7,bj5.edges,17467,10000,8
8,bj5.edges,17467,15000,1
9,bj5.edges,17467,15000,3
10,bj5.edges,8754,15000,3
11,bj5.edges,34152,15000,3
12,bj5.edges,65500,15000,3
13,bj5.edges,17467,15000,6
14,bj5.edges,17467,15000,8
15,bj5.edges,17467,1000,1
16,bj5.edges,17467,1000,3
17,bj5.edges,8754,1000,3
18,bj5.edges,34152,1000,3
19,bj5.edges,65500,1000,3
20,bj5.edges,17467,1000,6
21,bj5.edges,17467,1000,8
22,bj5.edges,17467,20000,1
23,bj5.edges,17467,20000,3
24,bj5.edges,8754,20000,3
25,bj5.edges,34152,20000,3
26,bj5.edges,65500,20000,3
27,bj5.edges,17467,20000,6
28,bj5.edges,17467,20000,8
29,bj5.edges,17467,25000,1
30,bj5.edges,17467,25000,3
31,bj5.edges,8754,25000,3
32,bj5.edges,34152,25000,3
33,bj5.edges,65500,25000,3
34,bj5.edges,17467,25000,6
35,bj5.edges,17467,25000,8
36,bj5.edges,17467,30000,1
37,bj5.edges,17467,30000,3
38,bj5.edges,8754,3000

In [ ]:
num_i = 0
for key, value in ins_list.items():
    info = key.split('-')
    road_map_code = info[0]
    if road_map_code == 'bj5':
        road_map = 'Beijing'
    elif road_map_code == 'cd1':
        road_map = 'Chengdu'
    else:
        road_map = 'Manhattan'
    a_row = road_map + ' & ' + info[1] + ' & ' + info[2]
    num_i += len(value)
    a_row = a_row + ' & ' + str(value)
    print(a_row)
print(count)
print(num_i)

In [5]:
def load_riders(f_name):
    riders = pd.read_csv(f_name, sep= ",", header= 0, index_col= False)
    riders.columns=["index", "s", "t", "et"]
    return riders

riders = load_riders('instances/instance_1-riders.csv')

In [8]:
def load_vehicles(f_name):
    vehicles = pd.read_csv(f_name, sep= ",", header= 0, index_col= False)
    vehicles.columns=["index","s","t", "p", "e", "c"]
    return vehicles

vehicles = load_vehicles('instances/instance_3-vehicles.csv')
vehicles.head(3)

,index,s,t,p,e,c
0,1,32306,32306,3,[0.29411765 0.52941176 0.71470588],3
1,2,98601,98601,3,[0.29411765 0.52941176 0.71470588],3
2,3,143505,143505,3,[0.29411765 0.52941176 0.71470588],3


In [77]:
test = ['1.', '', '', ' ', '', ' ', '1.8', '', '', '', '', '', '2.43', '', '', '', '', '2.916', '', '', '', '3.2805', '', '', '3.54294', '', '3.720087']
print([string for string in test if string != "" and string != " "])

['1.', '1.8', '2.43', '2.916', '3.2805', '3.54294', '3.720087']


In [28]:
n_gen = range(30, 50, 10)
objs = ['ew', 'et', 'ec', 'ed', 'se', 'sg',
        'ew,et,ec,ed,se,sg',
        'et,ec,ed,se,sg',
        'ew,ec,ed,se,sg',
        'ew,et,ed,se,sg',
        'ew,et,ec,se,sg',
        'ew,et,ec,ed,sg',
        'ew,et,ec,ed,se', 
        'ew,et,ec,ed']

batch_text_1 = ['#!/bin/bash\n', '#SBATCH --nodes=1     # Number of Tasks\n', '#SBATCH --ntasks=1\n', '#SBATCH --mem=250G\n', '#SBATCH --time=5:00:00\n', '\n', 'module load openmpi/3.0.0/intel\n', '\n', 'module load python/3.6.4\n', 'module load numpy/1.16.4\n', 'module load pandas\n', '\n']
batch_text_2 = ['#!/bin/bash\n', '#SBATCH --nodes=3     # Number of Tasks\n', '#SBATCH --ntasks=3\n', '#SBATCH --mem=150G\n', '#SBATCH --time=4:00:00\n', '\n', 'module load openmpi/3.0.0/intel\n', '\n', 'module load python/3.6.4\n', 'module load numpy/1.16.4\n', 'module load pandas\n', '\n']
count = 0
for i in range(1, 10):
    for j in n_gen:
        if j == 40:
            for o in objs:
                count += 1
                f_name = 'mny_rs/' + str(count) + '.sh'
                with open(f_name, 'w') as new_f:
                    new_f.writelines('%s' %line for line in batch_text_1)
                    com = 'mpirun -n 1 python ride_sharing.py -i ' + str(i) + ' -objs ' + o + ' -g ' + str(j) + ' -p 10'
                    print(count, com)
                    new_f.write(com)
        else:
            for o in objs:
                count += 1
                f_name = 'mny_rs/' + str(count) + '.sh'
                with open(f_name, 'w') as new_f:
                    new_f.writelines('%s' %line for line in batch_text_2)
                    com = 'mpirun -n 1 python ride_sharing.py -i ' + str(i) + ' -objs ' + o + ' -g ' + str(10) + ' -p 10 : -n 1 python ride_sharing.py -i ' + str(i) + ' -objs ' + o + ' -g ' + str(20) + ' -p 10 : -n 1 python ride_sharing.py -i ' + str(i) + ' -objs ' + o + ' -g ' + str(30) + ' -p 10'
                    print(count, com)
                    new_f.write(com)


1 mpirun -n 1 python ride_sharing.py -i 1 -objs ew -g 10 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ew -g 20 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ew -g 30 -p 10
2 mpirun -n 1 python ride_sharing.py -i 1 -objs et -g 10 -p 10 : -n 1 python ride_sharing.py -i 1 -objs et -g 20 -p 10 : -n 1 python ride_sharing.py -i 1 -objs et -g 30 -p 10
3 mpirun -n 1 python ride_sharing.py -i 1 -objs ec -g 10 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ec -g 20 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ec -g 30 -p 10
4 mpirun -n 1 python ride_sharing.py -i 1 -objs ed -g 10 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ed -g 20 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ed -g 30 -p 10
5 mpirun -n 1 python ride_sharing.py -i 1 -objs se -g 10 -p 10 : -n 1 python ride_sharing.py -i 1 -objs se -g 20 -p 10 : -n 1 python ride_sharing.py -i 1 -objs se -g 30 -p 10
6 mpirun -n 1 python ride_sharing.py -i 1 -objs sg -g 10 -p 10 : -n 1 python ride_sharing.py -i 1 -objs sg -g 20 -p 10 : -n 1

182 mpirun -n 1 python ride_sharing.py -i 7 -objs ew,et,ec,ed -g 10 -p 10 : -n 1 python ride_sharing.py -i 7 -objs ew,et,ec,ed -g 20 -p 10 : -n 1 python ride_sharing.py -i 7 -objs ew,et,ec,ed -g 30 -p 10
183 mpirun -n 1 python ride_sharing.py -i 7 -objs ew -g 40 -p 10
184 mpirun -n 1 python ride_sharing.py -i 7 -objs et -g 40 -p 10
185 mpirun -n 1 python ride_sharing.py -i 7 -objs ec -g 40 -p 10
186 mpirun -n 1 python ride_sharing.py -i 7 -objs ed -g 40 -p 10
187 mpirun -n 1 python ride_sharing.py -i 7 -objs se -g 40 -p 10
188 mpirun -n 1 python ride_sharing.py -i 7 -objs sg -g 40 -p 10
189 mpirun -n 1 python ride_sharing.py -i 7 -objs ew,et,ec,ed,se,sg -g 40 -p 10
190 mpirun -n 1 python ride_sharing.py -i 7 -objs et,ec,ed,se,sg -g 40 -p 10
191 mpirun -n 1 python ride_sharing.py -i 7 -objs ew,ec,ed,se,sg -g 40 -p 10
192 mpirun -n 1 python ride_sharing.py -i 7 -objs ew,et,ed,se,sg -g 40 -p 10
193 mpirun -n 1 python ride_sharing.py -i 7 -objs ew,et,ec,se,sg -g 40 -p 10
194 mpirun -n 1 py

In [34]:
n_gen = range(30, 50, 10)
objs = ['ew', 'et', 'ec', 'ed', 'se', 'sg',
        'ew,et,ec,ed,se,sg',
        'et,ec,ed,se,sg',
        'ew,ec,ed,se,sg',
        'ew,et,ed,se,sg',
        'ew,et,ec,se,sg',
        'ew,et,ec,ed,sg',
        'ew,et,ec,ed,se', 
        'ew,et,ec,ed']

batch_text_1 = ['#!/bin/bash\n', '#SBATCH --nodes=1     # Number of Tasks\n', '#SBATCH --ntasks=1\n', '#SBATCH --mem=250G\n', '#SBATCH --time=5:00:00\n', '\n', 'module load openmpi/3.0.0/intel\n', '\n', 'module load python/3.6.4\n', 'module load numpy/1.16.4\n', 'module load pandas\n', '\n']
batch_text_2 = ['#!/bin/bash\n', '#SBATCH --nodes=2     # Number of Tasks\n', '#SBATCH --ntasks=2\n', '#SBATCH --mem=150G\n', '#SBATCH --time=4:00:00\n', '\n', 'module load openmpi/3.0.0/intel\n', '\n', 'module load python/3.6.4\n', 'module load numpy/1.16.4\n', 'module load pandas\n', '\n']
count = 0
for i in range(1, 10):
    for j in n_gen:
        if j == 40:
            continue
        else:
            for k in range(int(len(objs)/2)):
                count += 1
                f_name = 'mny_rs/' + str(count) + '.sh'
                with open(f_name, 'w') as new_f:
                    new_f.writelines('%s' %line for line in batch_text_2)
                    com = 'mpirun -n 1 python ride_sharing.py -i ' + str(i) + ' -objs ' + objs[k] + ' -g ' + str(30) + ' -p 10 : -n 1 python ride_sharing.py -i ' + str(i) + ' -objs ' + objs[k+1] + ' -g ' + str(30) + ' -p 10 '
                    print(count, com)
                    new_f.write(com)



1 mpirun -n 1 python ride_sharing.py -i 1 -objs ew -g 30 -p 10 : -n 1 python ride_sharing.py -i 1 -objs et -g 30 -p 10 
2 mpirun -n 1 python ride_sharing.py -i 1 -objs et -g 30 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ec -g 30 -p 10 
3 mpirun -n 1 python ride_sharing.py -i 1 -objs ec -g 30 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ed -g 30 -p 10 
4 mpirun -n 1 python ride_sharing.py -i 1 -objs ed -g 30 -p 10 : -n 1 python ride_sharing.py -i 1 -objs se -g 30 -p 10 
5 mpirun -n 1 python ride_sharing.py -i 1 -objs se -g 30 -p 10 : -n 1 python ride_sharing.py -i 1 -objs sg -g 30 -p 10 
6 mpirun -n 1 python ride_sharing.py -i 1 -objs sg -g 30 -p 10 : -n 1 python ride_sharing.py -i 1 -objs ew,et,ec,ed,se,sg -g 30 -p 10 
7 mpirun -n 1 python ride_sharing.py -i 1 -objs ew,et,ec,ed,se,sg -g 30 -p 10 : -n 1 python ride_sharing.py -i 1 -objs et,ec,ed,se,sg -g 30 -p 10 
8 mpirun -n 1 python ride_sharing.py -i 2 -objs ew -g 30 -p 10 : -n 1 python ride_sharing.py -i 2 -objs et -g 30 -p 10

In [35]:
np.append([0,1], 9, 8, 7)

TypeError: _append_dispatcher() takes from 2 to 3 positional arguments but 4 were given